In [ ]:
import re
import csv
import unicodedata
from pathlib import Path

# ---------- CẤU HÌNH ----------
INPUT_FILE = r"E:\HistoryChatbot\data\raw\text\LSVN1954-1965.txt"

input_path = Path(INPUT_FILE)
processed_dir = input_path.parents[2] / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = processed_dir / f"{input_path.stem}_cleaned.txt"
LOG_FILE = processed_dir / f"{input_path.stem}_cleaning_log.csv"

PAGE_STOP = "15"  # xóa đến khi gặp dòng chỉ chứa số này

# Pattern tách page number lines
PAGE_NUM_LINE = re.compile(r'^\s*\d{1,4}\s*$')

# Pattern cho block chú thích (trích dẫn sách)
BIB_START = re.compile(r'^\s*\d+\.\s+.*?\b(Nxb\.?|, *tr\.\s*\d[\d\-\s]*)', re.IGNORECASE)
# BIB_START = re.compile(r'(Nxb\.?|, *tr\.\s*\d[\d\-\s]*)', re.IGNORECASE)
# Header normalized regex
HEADER_NORMALIZED_RE = re.compile(
    r'^\s*LICH\s*SU\s*VIET\s*NAM\s*[-–—]?\s*TAP\s*[0-9IVXLCDMIl\|]{1,4}\s*$',
    re.IGNORECASE
)

TITLE_RE = re.compile(r'(?i)\bCHƯƠNG\b|\bPHẦN\b|\bMỤC\b|\bCHƯƠNG\s+[IVXLCDM0-9]+\b')

MAX_HEADER_LEN = 120
MIN_NONSPACE_CHARS = 5


# ---------- HÀM TIỆN ÍCH ----------
def strip_accents(s: str) -> str:
    nfkd = unicodedata.normalize('NFKD', s)
    return ''.join([c for c in nfkd if not unicodedata.combining(c)])


def is_lichsu_header(line: str) -> bool:
    if not line or not line.strip():
        return False
    if len(line.strip()) > MAX_HEADER_LEN:
        return False
    norm = strip_accents(line).upper()
    norm = re.sub(r'\s+', ' ', norm).strip()
    return bool(HEADER_NORMALIZED_RE.match(norm))


def is_title_line(line: str) -> bool:
    if not line or not line.strip():
        return False
    norm = strip_accents(line).upper()
    return bool(TITLE_RE.search(norm))


def is_references_start(line: str) -> bool:
    if not line or not line.strip():
        return False
    norm = strip_accents(line).upper()
    norm = re.sub(r'\s+', ' ', norm).strip()
    return bool(re.search(r'\bTAI\s*LIEU\b.*\bTHAM\s*KHAO\b', norm))


def is_noise_line(line: str) -> bool:
    if not line or not line.strip():
        return False
    if is_lichsu_header(line) or is_title_line(line):
        return False

    s = line.strip()
    if re.match(r'^[\-\=\_\*\.\·\•\s]{1,}$', s):
        return True

    alnum_count = sum(1 for c in s if c.isalnum())
    if alnum_count < MIN_NONSPACE_CHARS:
        return True

    prop_alnum = alnum_count / max(1, len(s))
    if prop_alnum < 0.30:
        return True

    if len(s) < 30 and re.search(r'\s{3,}', line):
        return True

    return False


# KIỂM TRA TIÊU ĐỀ CHƯƠNG
def has_upper_block(lines: list[str], start: int, max_lines: int = 4) -> bool:
    for j in range(1, max_lines + 1):
        if start + j >= len(lines):
            break
        text = lines[start + j].strip()
        if not text:
            continue
        letters = [c for c in text if c.isalpha()]
        if letters:
            ratio_upper = sum(1 for c in letters if c.isupper()) / len(letters)
            if ratio_upper >= 0.7:
                return True
    return False


def should_keep_chapter(lines: list[str], i: int) -> bool:
    line = lines[i]
    if re.match(r'^\s*Chương\b', line, re.IGNORECASE):
        return has_upper_block(lines, i)
    return True  # không phải dòng "Chương"


# HÀNH ĐỘNG CHÍNH
def clean_book(input_path: str, output_path: Path, log_path: Path):
    raw = Path(input_path).read_text(encoding='utf-8')
    original_lines = raw.splitlines()
    removed_records = []

    cut_index = None
    for i, ln in enumerate(original_lines):
        if PAGE_NUM_LINE.match(ln) and ln.strip() == PAGE_STOP:
            cut_index = i
            break

    if cut_index is not None:
        removed_records.append({
            "line_no": 0,
            "original": f"removed all content before page {PAGE_STOP}",
            "reason": "REMOVE_FIRST_PAGES_UNTIL_15"
        })
        offset = cut_index + 1
        lines = original_lines[offset:]
    else:
        removed_records.append({
            "line_no": 0,
            "original": f"page marker {PAGE_STOP} not found; no front-pages removed",
            "reason": "NO_PAGE_MARKER"
        })
        offset = 0
        lines = original_lines

    cleaned = []
    i = 0
    while i < len(lines):
        ln = lines[i]
        orig_line_no = offset + i

        if is_references_start(ln):
            for k in range(i, len(lines)):
                removed_records.append({
                    "line_no": offset + k,
                    "original": lines[k],
                    "reason": "REFERENCES_SECTION_REMOVED"
                })
            break

        if is_lichsu_header(ln):
            removed_records.append({
                "line_no": orig_line_no,
                "original": ln,
                "reason": "LICH_SU_HEADER_REMOVED"
            })
            i += 1
            continue

        # loại bỏ trích dẫn sách
        if BIB_START.match(ln):
            block = [ln]
            j = i + 1
            while j < len(lines) and lines[j].strip() != "":
                block.append(lines[j])
                j += 1
            for k, b in enumerate(block):
                removed_records.append({
                    "line_no": offset + i + k,
                    "original": b,
                    "reason": "BIBLIOGRAPHIC_BLOCK_REMOVED"
                })
            i = j
            continue

        # xử lý "Chương" → giữ hoặc xóa
        if re.match(r'^\s*Chương\b', ln, re.IGNORECASE):
            if not should_keep_chapter(lines, i):
                removed_records.append({
                    "line_no": orig_line_no,
                    "original": ln,
                    "reason": "DUPLICATE_CHAPTER_HEADER_REMOVED"
                })
                i += 1
                continue

        cleaned.append(ln)
        i += 1

    intermediate = []
    for idx, ln in enumerate(cleaned):
        if PAGE_NUM_LINE.match(ln):
            removed_records.append({
                "line_no": offset + idx,
                "original": ln,
                "reason": "ISOLATED_PAGE_NUMBER_REMOVED"
            })
            continue
        intermediate.append(ln)

    final_lines = []
    prev_blank = False
    for idx, ln in enumerate(intermediate):
        original_ln_no = offset + idx
        if not ln.strip():
            if not prev_blank:
                final_lines.append('')
                prev_blank = True
            else:
                removed_records.append({
                    "line_no": original_ln_no,
                    "original": ln,
                    "reason": "EXCESSIVE_BLANK_REMOVED"
                })
            continue
        else:
            prev_blank = False

        if is_noise_line(ln):
            removed_records.append({
                "line_no": original_ln_no,
                "original": ln,
                "reason": "NOISE_LINE_REMOVED"
            })
            continue

        final_lines.append(ln)

    output_path.write_text("\n".join(final_lines), encoding='utf-8')

    with open(log_path, "w", encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["line_no", "original", "reason"], quoting=csv.QUOTE_ALL)
        writer.writeheader()
        for rec in removed_records:
            writer.writerow(rec)

    print("Done. Cleaned text saved to:", output_path)
    print("Log saved to:", log_path)


if __name__ == "__main__":
    clean_book(INPUT_FILE, OUTPUT_FILE, LOG_FILE)


Done. Cleaned text saved to: E:\HistoryChatbot\data\processed\LSVN1954-1965_cleaned.txt
Log saved to: E:\HistoryChatbot\data\processed\LSVN1954-1965_cleaning_log.csv


In [ ]:
import re
from typing import Iterable, Set, Tuple
from pathlib import Path

TR_PATTERN = re.compile(r'tr\.', re.IGNORECASE | re.UNICODE)


def _mark_window(index: int, total: int, neighbor: int) -> Iterable[int]:
    start = max(0, index - neighbor)
    end = min(total - 1, index + neighbor)
    return range(start, end + 1)


def clean_bibliographic_lines(text: str, neighbor: int = 2) -> Tuple[str, str]:
    """
    Xóa mọi dòng chứa 'tr.' (không phân biệt hoa/thường),
    đồng thời xóa luôn `neighbor` dòng trên và dưới (mặc định 2).
    Trả về tuple: (cleaned_text, removed_text).
    """
    if not text:
        return text, ""

    lines = text.splitlines()
    total = len(lines)
    to_delete: Set[int] = set()

    for i, line in enumerate(lines):
        if TR_PATTERN.search(line):
            for idx in _mark_window(i, total, neighbor):
                to_delete.add(idx)

    result_lines = [ln for i, ln in enumerate(lines) if i not in to_delete]
    removed_lines = [ln for i, ln in enumerate(lines) if i in to_delete]

    return "\n".join(result_lines), "\n".join(removed_lines)


def clean_file(input_path: str, output_path: str, removed_path: str, neighbor: int = 2) -> None:
    """
    Đọc văn bản từ file input_path, làm sạch,
    và ghi kết quả ra file output_path,
    đồng thời lưu các dòng bị xóa ra removed_path.
    """
    in_file = Path(input_path)
    if not in_file.exists():
        raise FileNotFoundError(f"Không tìm thấy file: {input_path}")

    text = in_file.read_text(encoding="utf-8")
    cleaned, removed = clean_bibliographic_lines(text, neighbor=neighbor)

    Path(output_path).write_text(cleaned, encoding="utf-8")
    Path(removed_path).write_text(removed, encoding="utf-8")


if __name__ == "__main__":
    # Ví dụ: đọc input.txt, ghi kết quả vào output.txt, dòng đã xóa vào removed.txt
    clean_file(r"E:\HistoryChatbot\data\processed\LSVN1954-1965_cleaned.txt", "output.txt", "removed.txt", neighbor=1)
    print("Đã xử lý xong.")
    print(" - Văn bản sạch: output.txt")
    print(" - Các dòng đã xóa: removed.txt")


Đã xử lý xong.
 - Văn bản sạch: output.txt
 - Các dòng đã xóa: removed.txt


In [ ]:
import re

def is_upper_block(lines: list[str], start: int, max_lines: int = 4) -> bool:
    """
    Kiểm tra sau 'Chương' có block full chữ hoa không.
    """
    for j in range(1, max_lines + 1):
        if start + j >= len(lines):
            break
        text = lines[start + j].strip()
        if not text:
            continue
        letters = [c for c in text if c.isalpha()]
        if letters:
            ratio_upper = sum(1 for c in letters if c.isupper()) / len(letters)
            if ratio_upper >= 0.7:
                return True
    return False



def check_chapter_block(text: str) -> str:
    """
    Trả về:
      - 'KEEP' nếu là tiêu đề chương gốc (sau 'Chương' có block chữ hoa),
      - 'REMOVE' nếu chỉ là header lặp,
      - 'NO_CHAPTER_FOUND' nếu không có 'Chương'.
    """
    lines = text.splitlines()
    for i, line in enumerate(lines):
        if re.match(r'^\s*Chương\b', line, re.IGNORECASE):
            return "KEEP" if is_upper_block(lines, i) else "REMOVE"
    return "NO_CHAPTER_FOUND"


# ----------------- DEMO -----------------
if __name__ == "__main__":
    text = """
Chương I 

“MIỄN BÁC TRONG THỜI KỲ 
KHÔI PHỤC, CẢI TẠO VÀ BƯỚC ĐÀU 
PHÁT TRIẾN KINH TÉ, VĂN HÓA 
(1954-1960) 
    """
    result = check_chapter_block(text)
    print(result)  # -> "KEEP"


KEEP


In [ ]:
def normalize_newlines(input_file: str, output_file: str) -> None:
    """
    Chuẩn hóa file văn bản: bỏ xuống dòng lộn xộn,
    chỉ giữ khoảng trắng khi là ngắt đoạn.
    """
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    # Thay nhiều khoảng trắng dòng -> 2 xuống dòng
    import re
    text = re.sub(r"[ \t]+", " ", text)  # gộp khoảng trắng thừa
    text = re.sub(r"\n{3,}", "\n\n", text)  # >2 xuống dòng -> 2
    text = re.sub(r"([^\n])\n([^\n])", r"\1 \2", text)  # nối dòng ngắt sai

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)


if __name__ == "__main__":
    input_path = r"E:\HistoryChatbot\data\processed\LSVN1954-1965_cleaned_ver2.txt"
    output_path = "output.txt"
    normalize_newlines(input_path, output_path)
    print(f"Đã xử lý xong. Kết quả lưu tại: {output_path}")


Đã xử lý xong. Kết quả lưu tại: output.txt


### Một số vấn đề khi lưu lại dữ liệu vào json:
- đang bị nhầm lẫn các tiêu đề ở phần subsection, section, chapter, hiện tại đang bị nhầm lẫn ở 2 chỗ:
    - Nhầm lẫn với phần trích dẫn sách còn sót lại sau khi xử lý dữ liệu => khả năng vẫn xóa hết phần trích dẫn nào có tr...
    - Nhầm lẫn với một vài đoạn văn bản mà đầu văn bản được đưa bởi số liệu
- Chapter đang bị lặp bởi trong sách nó nằm ở phần tiêu đề phía trên ở mỗi trang(có thể xóa được, tuy nhiên cần phải biết được xóa ở đâu để tránh nhầm lẫn với tiêu đề Chương gốc), nếu sau Chương.. là một đoạn text full chữ hoa thì giữ lại, là tiêu đề chương, còn lại là nhiễu
(5068)(5217)
- trước khi xóa có lẽ cần co dòng lại cho hợp format
- REFERENCES_SECTION_REMOVED đang bị xóa nhầm cả với các tiêu đề thay vì chỉ xóa mình chú thích